# Distances

In [ ]:
from ipynb.fs.full.koselleck import *

In [ ]:
FORCE=False
GBY_LOCAL_O=['corpus1', 'corpus2','period1', 'period2', 'word1', 'word2','qstr']

## Generating distances en masse

In [ ]:
def _distvecs(objd): return distvecs(**objd)

def distvecs(period=None,run=None,prefix='dvecs',max_num=10000,num_runs=10,
             num_proc=1,force=False,progress=True,cache_only=True,cache=True):
    
    argd=dict(
        period=period,run=run,prefix=prefix,
        max_num=max_num,num_runs=num_runs,
        num_proc=num_proc,force=force,progress=progress,
        cache_only=cache_only,cache=cache
    )
    odf=None
    
    if period is None:
        # load cached word?
        objs=[{**argd, **{'period':prd, 'progress':True, 'num_proc':1,'cache_only':True}} for prd in get_default_periods()]
        odf=pd.concat(pmap(_distvecs, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress))
        return odf if not cache_only else pd.DataFrame()
    
    if run is None:
        objs=[{**argd, **{'run':run+1, 'progress':True, 'num_proc':1,'cache_only':True}} for run in range(num_runs)]
        odf=pd.concat(pmap(_distvecs, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress))
        return odf if not cache_only else pd.DataFrame()

    # load vecs?
    dfvecs=vecs(period=period, run=run)
    if not len(dfvecs): return pd.DataFrame()
    
    pprefix=prefix+'_'+period.split('-')[0]+'_'+str(run).zfill(2)
#     ofnfn=os.path.join(PATH_DB,'dvecs',f'data.dvecs.{period}_{run:02}.pkl')
#     qstr=f'{period}_{run},max_num={max_num}'
#     if cache and not force:
# #         if os.path.exists(ofnfn):
# #             print('Found on disk. Loading...')
#             #with open(ofnfn,'rb') as f: od=pickle.load(f)
# #             odf=read_df(ofnfn)
# #             print('Finished loading')
# #             return odf if not cache_only else pd.DataFrame()
#         with get_veclib(pprefix,autocommit=False) as vl:
#             if len()
#             if qstr in vl:
#                 print('Found in vector library. Loading')
#                 odf=vl[qstr] if not cache_only else pd.DataFrame()
#                 print('Finished loading')
#                 return odf
    # filter
    dfvecs=dfvecs.iloc[:max_num]
    
    #distmat!
    if progress: print(f'Computing distances for {max_num} words')
    dfdist=pd.DataFrame(
        fastdist.cosine_pairwise_distance(
            dfvecs.values.astype(float),
            return_matrix=True
        ),
        index=dfvecs.index,
        columns=dfvecs.index
    )
    # csim to cdist?
    dfdist=2-dfdist
    if progress: print('Done')
    
    
    
    # convert to long form
#     od = defaultdict(dict)
# #     g=nx.Graph()
    if cache:
#         with get_veclib(pprefix,autocommit=False) as vl:
#             for i,w1 in enumerate(tqdm(dfdist.index,position=0,disable=not progress)):
#                 vl[w1]=dfdist.loc[w1]
# #                 if i and not i%100: vl.commit()
#             vl.commit()
        def tryword(w1):
            try:
                with get_veclib(prefix=w1,folders=['wvecs'],autocommit=False) as vl:
                    qstr=f'{period}_{run:02}'
                    if force or qstr not in vl:
                        vl[qstr]=dfdist.loc[w1]
                        vl.commit()
            except Exception as e:
                print('!!',e)
                return w1
            return 0

        tryagain=[]
        for i,w1 in enumerate(tqdm(dfdist.index,position=0,disable=not progress)):
            res=tryword(w1)
            if res!=0: tryagain+=[w1]
        for i,w1 in enumerate(tqdm(tryagain,position=0,disable=not progress)):
            res=tryword(w1)
        
        
                
            
#             if w1<w2:
# #                 g.add_edge(w1,w2,cdist=val)
#                 od[w1][w2]=val
#     if cache:
#         dfdist.to_pickle(ofnfn)
        #with open(ofnfn,'wb') as of: pickle.dump(od,of)
#         with get_veclib(pprefix,autocommit=True) as vl:
#             vl[qstr]=od
    return dfdist if not cache_only else pd.DataFrame()

In [ ]:
# res=distvecs('1780-1785',1)#.loc['culture'][['culture','represent']]
# res

In [ ]:
# res=distvecs('1780-1785',num_proc=4)#.loc['culture'][['culture','represent']]
# res

In [ ]:
for prd in get_default_periods():
    if prd<"1750": continue
    print(prd)
    distvecs(prd,num_proc=1,cache=True,cache_only=True,force=False)

In [ ]:
print('Done!')